# EasyModel Backend on Google Colab - 100% FREE! 🚀

Run your ML training backend with **FREE GPU/CPU** on Google Colab!

## Quick Setup:
1. Click **Runtime → Run all**
2. Copy the ngrok URL that appears
3. Set it in Vercel: `NEXT_PUBLIC_EASYMODEL_API_URL=<your-ngrok-url>`
4. Also set: `EASYMODEL_API_URL=<your-ngrok-url>`
5. Redeploy Vercel
6. Train models for FREE at easymodel.vercel.app!


In [ ]:
# Step 1: Install dependencies
print("📦 Installing dependencies...")
%pip install -q fastapi uvicorn python-dotenv transformers torch datasets huggingface-hub pyngrok structlog
print("✅ Dependencies installed!")


In [ ]:
# Step 2: Clone your repository
print("📥 Cloning repository...")
!git clone https://github.com/lehendo/easymodel.git
%cd easymodel
print("✅ Repository cloned!")


In [ ]:
# Step 3: Set up your tokens
# Get ngrok token from: https://dashboard.ngrok.com/get-started/your-authtoken
# Get HF token from: https://huggingface.co/settings/tokens

NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"
HF_TOKEN = "YOUR_HF_TOKEN_HERE"

# Or use Colab Secrets (left sidebar → key icon → add NGROK_TOKEN and HF_TOKEN)
try:
    from google.colab import userdata
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✅ Using tokens from Colab secrets")
except:
    print("⚠️  Using tokens from code (not secure for public notebooks!)")

import os
os.environ['HUGGINGFACE_API_KEY'] = HF_TOKEN
os.environ['HF_TOKEN'] = HF_TOKEN

print("🔑 Configuring ngrok...")
!ngrok config add-authtoken {NGROK_TOKEN}
print("✅ Tokens configured!")


In [ ]:
# Step 4: Update CORS and disable wandb (experiment tracking)
print("🔧 Updating CORS settings and disabling wandb...")

import os
# Disable wandb (Weights & Biases) to avoid interactive prompts
os.environ['WANDB_DISABLED'] = 'true'
os.environ['WANDB_MODE'] = 'disabled'

with open('main.py', 'r') as f:
    content = f.read()

# Replace the allowed_origins line
import re
content = re.sub(
    r'allowed_origins = os\.getenv\(.*?\)\.split\(.*?\)',
    'allowed_origins = ["*"]  # Allow all for demo',
    content
)

with open('main.py', 'w') as f:
    f.write(content)

print("✅ CORS updated and wandb disabled!")


In [ ]:
# Step 5: Start the backend with ngrok
from pyngrok import ngrok
from pyngrok.conf import PyngrokConfig
import subprocess
import time
import threading

print("🚀 Starting backend server...")

# Configure ngrok to skip browser warning (helps with EventSource/SSE)
pyngrok_config = PyngrokConfig(inspect_enabled=False)

# Start ngrok tunnel
public_url = ngrok.connect(8000, pyngrok_config=pyngrok_config, bind_tls=True)

print("\n" + "="*70)
print("🎉 YOUR BACKEND IS LIVE!")
print("="*70)
print(f"\n📡 Public URL: {public_url}")
print("\n📋 NEXT STEPS:")
print("   1. Copy the URL above (just the https://... part)")
print("   2. Go to: https://vercel.com → Your Project → Settings")
print("   3. Environment Variables → Add/Edit BOTH:")
print(f"      NEXT_PUBLIC_EASYMODEL_API_URL={public_url}")
print(f"      EASYMODEL_API_URL={public_url}")
print("   4. Deployments → Redeploy (uncheck 'Use existing Build Cache')")
print("   5. Visit easymodel.vercel.app and TRAIN FOR FREE! 🎊")
print("\n⚠️  IMPORTANT: Keep this notebook tab open while training")
print("="*70 + "\n")

# Start FastAPI server
!uvicorn main:app --host 0.0.0.0 --port 8000
